# Introducción

En este trabajo se utilizó `fasttext` para resolver un problema de NLI: Entrenar un predictor capaz de clasificar frases en 3 posibles clases: `entailment`, `contradiction` y `neutral`. Este problema se encuentra presentado en [Gururangan et al., 2018][1].
A continuación se presenta el modelo implementado.


[1]:https://www.aclweb.org/anthology/N18-2017

# Generación de datos

Mediante esta función se toman los archivos de oraciones y etiquetas, y se genera un nuevo archivo en el formato aceptado por `fasttext`.

De acuerdo con el formato de `fasttext`, el archivo de entrenamiento debe estar constituido en cada línea por la oración que se desea aprender, y a continuación su correspondiente etiqueta predecida por el prefijo `__label__`. Por ejemplo:
```
A child is reaching to touch the propeller out of curiosity. __label__contradiction
A child is playing with a ball. __label__entailment
A woman is doing a cartwheel. __label__entailment
A woman is fixing her home. __label__contradiction
A woman is doing a cartwheel and falls on her head. __label__neutral
```

Por otra parte, en caso de que el archivo de etiquetas no sea pasado como argumento, el archivo generado contendrá una oración por cada renglón.


In [ ]:
import json
import csv
import fasttext

def read_data(sentences_path, results_path, labels_path=None):
    sentence_data = open(sentences_path, 'r')
    result_file = open(results_path, 'w')
    
    def it_sentences(sentence_data):
        for line in sentence_data:
            example = json.loads(line)
            yield example['sentence2']

    def it_labels(label_data):
        label_data_reader = csv.DictReader(label_data)
        for example in label_data_reader:
            yield example['gold_label']
    
    if labels_path:
        label_data = open(labels_path, 'r')
        for sentence, label in zip(it_sentences(sentence_data), it_labels(label_data)):
            print(f"{sentence} __label__{label}")
            result_file.write(f"{sentence} __label__{label}\n")
    else:
        for sentence in it_sentences(sentence_data):
            print(sentence)
            result_file.write(f"{sentence}\n")
    
    result_file.close()

El primer paso para realizar el entrenamiento de la red consiste en transformar los archivos de datos al formato aceptado por `fasttext`. Para esto se debe utilizar la función del bloque anterior.

In [ ]:
# Generación de archivos de train
train_sentences_path = '../input/snli_1.0_train_filtered.jsonl'
train_labels_path = '../input/snli_1.0_train_gold_labels.csv'
train_file_path = 'train_data.txt'

read_data(
    sentences_path=train_sentences_path, 
    labels_path=train_labels_path,
    results_path=train_file_path)


In [ ]:
# Generación de archivos de validacion
val_sentences_path = '../input/snli_1.0_dev_filtered.jsonl'
val_labels_path = '../input/snli_1.0_dev_gold_labels.csv'
val_file_path = 'val_data.txt'

read_data(
    sentences_path=val_sentences_path, 
    labels_path=val_labels_path,
    results_path=val_file_path)

In [ ]:
# Generación de archivos de test
test_sentences_path = '../input/snli_1.0_test_filtered.jsonl'
test_file_path = 'test_data.txt'

read_data(
    sentences_path=test_sentences_path, 
    results_path=test_file_path)

Los datos de entrenamiento, de validación y de testeo se guardan en los archivos `train_data.txt`, `val_data.txt` y `test_data.txt` respectivamente.

# Configuración de hiperparámetros

De acuerdo con la API de `fasttext`, para el aprendizaje supervisado se tienen los siguientes hiperparámetros para configurar:
```
    input             # training file path (required)
    lr                # learning rate [0.1]
    dim               # size of word vectors [100]
    ws                # size of the context window [5]
    epoch             # number of epochs [5]
    minCount          # minimal number of word occurences [1]
    minCountLabel     # minimal number of label occurences [1]
    minn              # min length of char ngram [0]
    maxn              # max length of char ngram [0]
    neg               # number of negatives sampled [5]
    wordNgrams        # max length of word ngram [1]
    loss              # loss function {ns, hs, softmax, ova} [softmax]
    bucket            # number of buckets [2000000]
    thread            # number of threads [number of cpus]
    lrUpdateRate      # change the rate of updates for the learning rate [100]
    t                 # sampling threshold [0.0001]
    label             # label prefix ['__label__']
    verbose           # verbose [2]
    pretrainedVectors # pretrained word vectors (.vec file) for supervised learning []
```
A continuación se deben configurar aquellos parámetros que resulten diferentes a los que vienen por defecto con el objetivo de realizar el entrenamiento del modelo.

In [ ]:
# import fasttext
# Ejecutar estas líneas para cargar alguno de los modelos ya entrenados:
# model = fasttext.load_model("model_prueba1_filename.bin")
# model = fasttext.load_model("model_prueba2_filename.bin")
# model = fasttext.load_model("model_prueba3_filename.bin")

# Entrenamiento

Durante la elección de los hiperparámetros del modelo se vio que lo que más afectaba al porcentaje de aciertos en el conjunto de validación era el valor del *learning rate* y del *learning rate update rule*. De esta manera, eligiendo un modelo N-gramas de mayor orden (N=3,4,...) los resultados se muestran casi invariantes, con lo cual se optó por mantener un modelo de bigramas. También se probó cambiar el número de palabras en el contexto utilizado, pero el porcentaje de aciertos en validación no sube de 67,2% para un contexto de tres palabras o más. Por último, la función de costo que mejor resultados da es *One-vs-all*.

In [75]:
# Hiperparámetros elegidos
num_epochs = 50 # Número de epochs
learning_rate = 0.001 # Tasa de aprendizaje
lr_update_rate = 10 # Tasa de reducción del learning rate
embedding_size = 40 # Dimensión del espacio de los embeddings
context_window_size = 3 # Cantidad de palabras en un contexto
wordNgrams = 2 # Bigrama
loss = "ova" # Función de costo a utilizar

In [76]:
# Archivo de entrenamiento
train_file_path = 'train_data.txt'

# Entrenamiento del modelo
model = fasttext.train_supervised(
    input=train_file_path, 
    epoch=num_epochs, 
    lr=learning_rate,
    lrUpdateRate=lr_update_rate,
    dim=embedding_size,
    ws=context_window_size,
    wordNgrams=wordNgrams,
    loss=loss
)

In [ ]:
# Ruta al archivo del modelo
model_path = "model_filename.bin"
# Se guarda el modelo entrenado para poder ser restaurado luego
model.save_model(model_path)

# Validación

In [77]:
# Ruta al archivo de validación
val_file_path = 'val_data.txt'

# Función encargada de imprimir los resultados de la validación
def print_results(N, p, r):
    print("N\t" + str(N))
    print("Prediction @{}\t{:.3f}".format(1, p))
    print("Recall @{}\t{:.3f}".format(1, r))

# Validación del modelo entrenado
print_results(*model.test(val_file_path))

N	9842
Prediction @1	0.672
Recall @1	0.672


# Testeo

In [ ]:
# Ruta al archivo de testeo
test_file_path = 'test_data.txt'

# Realización de una predicción del modelo entrenado
model.predict("boy leaving baseball game")

# Generación del archivo de predicciones

Un vez entrenado el modelo, se debe exportar un archivo que contenga todas las predicciones del set de testeo, de manera de comprobar la generalización. El archivo de salida consistirá en un `txt` que en cada línea contiene la clase predicha.

In [78]:
# Archivo de entrada con las oraciones que se desean clasificar
test_file_path = 'test_data.txt'
# Archivo de salida que contendrá las clases predichas
test_labels_filename = "test_cls.txt"

predictions_file = open(test_labels_filename, "w")
with open(test_file_path, 'r') as fout:
    for line in fout.readlines():
        prediction = model.predict(line.rstrip('\n'))[0][0]
        predictions_file.write(f"{prediction}\n")
        print(prediction)
predictions_file.close()

__label__contradiction
__label__neutral
__label__contradiction
__label__entailment
__label__neutral
__label__neutral
__label__entailment
__label__neutral
__label__entailment
__label__neutral
__label__entailment
__label__entailment
__label__neutral
__label__entailment
__label__contradiction
__label__contradiction
__label__contradiction
__label__neutral
__label__contradiction
__label__contradiction
__label__contradiction
__label__contradiction
__label__entailment
__label__neutral
__label__neutral
__label__entailment
__label__entailment
__label__entailment
__label__neutral
__label__contradiction
__label__entailment
__label__entailment
__label__entailment
__label__contradiction
__label__neutral
__label__contradiction
__label__contradiction
__label__entailment
__label__contradiction
__label__entailment
__label__entailment
__label__contradiction
__label__contradiction
__label__contradiction
__label__contradiction
__label__neutral
__label__entailment
__label__entailment
__label__neutral
__lab

__label__neutral
__label__neutral
__label__contradiction
__label__contradiction
__label__contradiction
__label__contradiction
__label__neutral
__label__contradiction
__label__contradiction
__label__entailment
__label__neutral
__label__contradiction
__label__entailment
__label__entailment
__label__entailment
__label__contradiction
__label__entailment
__label__neutral
__label__contradiction
__label__neutral
__label__neutral
__label__neutral
__label__neutral
__label__contradiction
__label__entailment
__label__contradiction
__label__neutral
__label__neutral
__label__entailment
__label__contradiction
__label__contradiction
__label__contradiction
__label__neutral
__label__neutral
__label__neutral
__label__contradiction
__label__contradiction
__label__entailment
__label__neutral
__label__neutral
__label__contradiction
__label__entailment
__label__contradiction
__label__contradiction
__label__contradiction
__label__contradiction
__label__entailment
__label__neutral
__label__neutral
__label__en

__label__neutral
__label__neutral
__label__contradiction
__label__contradiction
__label__neutral
__label__entailment
__label__entailment
__label__entailment
__label__entailment
__label__contradiction
__label__neutral
__label__contradiction
__label__contradiction
__label__entailment
__label__neutral
__label__neutral
__label__contradiction
__label__entailment
__label__contradiction
__label__neutral
__label__contradiction
__label__neutral
__label__neutral
__label__contradiction
__label__entailment
__label__contradiction
__label__contradiction
__label__neutral
__label__entailment
__label__contradiction
__label__contradiction
__label__contradiction
__label__neutral
__label__contradiction
__label__contradiction
__label__neutral
__label__entailment
__label__contradiction
__label__entailment
__label__entailment
__label__entailment
__label__neutral
__label__contradiction
__label__entailment
__label__neutral
__label__entailment
__label__entailment
__label__entailment
__label__entailment
__label_

__label__neutral
__label__neutral
__label__entailment
__label__neutral
__label__entailment
__label__neutral
__label__contradiction
__label__neutral
__label__entailment
__label__neutral
__label__contradiction
__label__entailment
__label__contradiction
__label__contradiction
__label__contradiction
__label__neutral
__label__contradiction
__label__contradiction
__label__contradiction
__label__entailment
__label__contradiction
__label__entailment
__label__neutral
__label__contradiction
__label__entailment
__label__entailment
__label__neutral
__label__entailment
__label__neutral
__label__entailment
__label__neutral
__label__contradiction
__label__entailment
__label__entailment
__label__contradiction
__label__neutral
__label__neutral
__label__entailment
__label__contradiction
__label__neutral
__label__neutral
__label__entailment
__label__entailment
__label__neutral
__label__entailment
__label__neutral
__label__contradiction
__label__entailment
__label__entailment
__label__neutral
__label__con

__label__contradiction
__label__neutral
__label__contradiction
__label__contradiction
__label__contradiction
__label__neutral
__label__entailment
__label__entailment
__label__entailment
__label__contradiction
__label__neutral
__label__entailment
__label__contradiction
__label__contradiction
__label__contradiction
__label__neutral
__label__contradiction
__label__neutral
__label__entailment
__label__entailment
__label__entailment
__label__entailment
__label__entailment
__label__entailment
__label__entailment
__label__neutral
__label__contradiction
__label__contradiction
__label__neutral
__label__contradiction
__label__neutral
__label__neutral
__label__entailment
__label__entailment
__label__contradiction
__label__contradiction
__label__contradiction
__label__neutral
__label__neutral
__label__neutral
__label__neutral
__label__entailment
__label__neutral
__label__entailment
__label__neutral
__label__entailment
__label__neutral
__label__contradiction
__label__entailment
__label__contradicti

__label__neutral
__label__neutral
__label__contradiction
__label__neutral
__label__contradiction
__label__contradiction
__label__neutral
__label__contradiction
__label__neutral
__label__contradiction
__label__contradiction
__label__entailment
__label__entailment
__label__entailment
__label__neutral
__label__contradiction
__label__contradiction
__label__neutral
__label__contradiction
__label__neutral
__label__neutral
__label__contradiction
__label__entailment
__label__contradiction
__label__neutral
__label__contradiction
__label__entailment
__label__entailment
__label__entailment
__label__contradiction
__label__neutral
__label__entailment
__label__contradiction
__label__entailment
__label__contradiction
__label__contradiction
__label__neutral
__label__entailment
__label__contradiction
__label__entailment
__label__contradiction
__label__entailment
__label__contradiction
__label__neutral
__label__entailment
__label__neutral
__label__entailment
__label__contradiction
__label__contradiction

Una vez generadas las etiquetas de salida, se debe unificar todo en un único archivo para poder ser evaluado en la competencia de kaggle. La función a continuación se encarga de esta tarea.

In [79]:
import argparse
import json
import csv

# Función encargada de transformar las predicciones al formato necesario para ser evaluadas en kaggle
def format_predictions(sentences_filename, labels_filename, output_filename):
    with open(output_filename, 'w') as fout:
        csv_writer = csv.writer(fout)
        csv_writer.writerow(['pairID', 'gold_label'])

        for pairID, label in it_ID_label_pairs(sentences_filename, labels_filename):
            formatted_label = format_label(label)
            csv_writer.writerow([pairID, formatted_label])

def format_label(label):
    return label[len("__label__"):]

def it_ID_label_pairs(sentences_filename, labels_filename):
    sentence_data = open(sentences_filename, 'r')
    labels_data = open(labels_filename, 'r')
    for pairID, label in zip(it_ID(sentence_data), it_labels(labels_data)):
        yield pairID, label

def it_ID(sentence_data):
    for line in sentence_data:
        example = json.loads(line)
        yield example['pairID']

def it_labels(label_data):
    for label in label_data:
        label = label.rstrip('\n')  # sacamos el fin de linea
        yield label

In [ ]:
sentences_filename = "../input/snli_1.0_test_filtered.jsonl"
labels_filename = "test_cls.txt"
output_filename = "result.csv"

# Exportación de los resultados del testeo en el formato correcto
format_predictions(sentences_filename, labels_filename, output_filename)

In [80]:
# Se imprimen los resultados almacenados en el archivo de salida
!cat result.csv